In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import time 
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [7]:
def readucr(filename):#读取数据
    data = np.loadtxt(filename, dtype=str, delimiter = ',')
    x = np.array(data[:, 0:-2], dtype=np.float32)
    y = np.array(data[:, 1:-1], dtype=np.float32)
    x = x.reshape((x.shape[0],x.shape[1],1))
    y = y.reshape((y.shape[0],y.shape[1],1))
    print(f'{filename}: {x.shape}, {y.shape}')
    return x, y

In [3]:
def Data_MinMax_Scaler(x_train):# 归一化
    sample_num = x_train.shape[0]
    for i in range(0, sample_num): 
        scaler = MinMaxScaler(feature_range=(-1, 1)) # 归一化到 [-1, 1] 区间内
        x_train[i] = scaler.fit_transform(x_train[i]) # fit 获得最大值和最小值，transform 执行归一化
    return x_train

In [4]:
class Autoencoder_GRU: # GRU神经网络
    """
        model初始化
    """
    def __init__(self, output_directory, input_shape, output_shape, verbose= True): 
        self.output_directory = output_directory
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.verbose = verbose
        
        self.model = self.build_model() # 调用 build_model
        if(verbose == True):
            self.model.summary() # 打印 model 结构
        if not os.path.exists(self.output_directory):
            os.mkdir(self.output_directory)
        init_model_file = os.path.join(self.output_directory, 'model_init.hdf5')
        self.model.save_weights(init_model_file) # 保存初始权重

    """
       创建model
    """         
    def build_model(self): 
        # model 网络结构
        input_layer = tf.keras.Input(self.input_shape)
        layer_1 = tf.keras.layers.GRU(20, return_sequences=True)(input_layer) 
        layer_2 = tf.keras.layers.BatchNormalization()(layer_1)
        layer_3 = tf.keras.layers.Activation(activation='tanh', name='gru_output')(layer_2)
        output_layer = tf.keras.layers.Dense(self.output_shape)(layer_3)

        model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
        # 编译model
        model.compile(loss='mse', optimizer='adam',metrics=['mae']) 
        
        # 设置 callbacks 回调
        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', # 用于动态调整learning rate
                                                         factor=0.5, 
                                                         patience=50, 
                                                         min_lr=0.0001)

        file_path =  os.path.join(self.output_directory, 'best1_model.hdf5')

        model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=file_path, 
                                                              monitor='loss', 
                                                              save_best_only=True)

        self.callbacks = [reduce_lr, model_checkpoint]

        return model 

    # 训练model
    def fit(self, x_train, y_train, x_val, y_val, epochs): 
        """
            验证集: (x_val, y_val) 用于监控loss，防止overfitting
        """  
        batch_size = 16
        mini_batch_size = int(min(x_train.shape[0]/10, batch_size))

        start_time = time.time() 

        hist = self.model.fit(x_train, y_train, 
                              validation_data=(x_val, y_val),
                              batch_size=mini_batch_size, 
                              epochs=epochs,
                              verbose=self.verbose,  
                              callbacks=self.callbacks)
        
        duration = time.time() - start_time

        tf.keras.backend.clear_session() # 清除当前tf计算图
        

In [5]:
def output_of_gru(train_data, filepath, timewindows=1):#提取Gru的输出
    '''
    @train_data:模型的输入，例如Car_TRAIN.txt
    @filepath:训练好的模型参数
    @timewindows:时间窗
    '''
    initial_model = tf.keras.models.load_model(filepath)
    
    gru_output_model = tf.keras.Model(
    inputs = initial_model.input,
    outputs = initial_model.get_layer('gru_output').output)
    
    output = gru_output_model.predict(train_data)
    
    return np.array(output[:, ::timewindows])

# 测试output_of_gry的功能

In [8]:
X_train,Y_train = readucr('dataset/Car/Car_TRAIN.txt')
X_test,Y_test = readucr('dataset/Car/Car_TEST.txt')

dataset/Car/Car_TRAIN.txt: (60, 575, 1), (60, 575, 1)
dataset/Car/Car_TEST.txt: (60, 575, 1), (60, 575, 1)


In [9]:
X_train = Data_MinMax_Scaler(X_train)
Y_train = Data_MinMax_Scaler(Y_train)
X_test = Data_MinMax_Scaler(X_test)
Y_test = Data_MinMax_Scaler(Y_test)

In [ ]:
# 划分 训练集 和 验证集
x_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, 
                                                      test_size=0.33, 
                                                      shuffle=True,
                                                      random_state=42)

In [18]:
output = output_of_gru(X_train,'configuration/V1/testbest1_model.hdf5',5)

In [19]:
output.shape

(60, 115, 20)